In [1]:
import ftplib
import time as tm
import os

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import librosa
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import IPython.display as ipd

In [3]:
from twilio.rest import Client
sid = 'ACdd02d37c618e196bf7d73790984b4797'
authToken = 'bfbc7af7aacbfd63da6470601b06d383'
client = Client(sid, authToken)

In [4]:
model = load_model('Model.h5')

final = pd.read_pickle("extracted_df.pkl")
y = np.array(final["class"].tolist())
le = LabelEncoder()
to_categorical(le.fit_transform(y))


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [5]:
# funcion de prediccion
def model_Prediction(file_name):
    # carga el erchivo
    audio_data, sample_rate = librosa.load(file_name, res_type="kaiser_fast")
    # obtiene las caracteristicas MFCC
    feature = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=128)
    # escalado
    feature_scaled = np.mean(feature.T, axis=0)
    # lista las caracteristicas
    prediction_feature = np.array([feature_scaled])
    # redimenciona
    final_prediction_feature = prediction_feature.reshape(
        prediction_feature.shape[0], 16, 8, 1
    )
    # obtiene la id de la clase usando argmax
    predicted_vector = np.argmax(model.predict(final_prediction_feature), axis=-1)
    #obtiene la categoria de la clase desde la id
    predicted_class = le.inverse_transform(predicted_vector)
    # resultado
    print("El modelo a predicho la clase como  --> ", predicted_class[0])

    return predicted_class[0]

In [6]:
ftp = ftplib.FTP('ftp-serveralways.alwaysdata.net', 'serveralways', 'Always322981')
ftp.pwd()
ftp.cwd("audios")
ftp.retrlines('LIST')

-rw-rw-r--   1 serveralways 337638     160044 Nov 28 13:12 mic_20231128_13_12_20.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec  1 02:05 mic_20231201_02_05_29.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec  1 02:06 mic_20231201_02_05_48.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec  1 02:44 mic_20231201_02_44_45.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec  1 02:45 mic_20231201_02_45_03.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec  1 02:45 mic_20231201_02_45_23.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec 18 20:58 mic_20231218_20_58_09.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec 23 19:11 mic_20231223_19_11_15.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec 23 19:11 mic_20231223_19_11_33.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec 23 19:12 mic_20231223_19_12_19.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec 23 19:13 mic_20231223_19_13_25.wav
-rw-rw-r--   1 serveralways 337638     160044 Dec 23 19:13 mic_20

'226 Transfer complete'

In [12]:
ftp.retrlines('LIST')

-rw-rw-r--   1 serveralways 337638     160044 Jan 12 14:10 mic_20240112_14_10_44.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 12 14:11 mic_20240112_14_11_02.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 12 14:11 mic_20240112_14_11_20.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 12 14:11 mic_20240112_14_11_37.wav


'226 Transfer complete'

In [11]:
archivos = ftp.nlst()
num = len(archivos)
num

4

In [16]:
ftp.sendcmd("TYPE I")
size = ftp.size(archivos[0])
print(size)

160044


In [8]:
file = archivos[0]

tiempo = tm.localtime()
audio_name = 'mic_'+str(tiempo[0])+str("%02d" % tiempo[1])+str("%02d" % tiempo[2])+'_'+str("%02d" % tiempo[3])+'_'+str("%02d" % tiempo[4])+'_'+str("%02d" % tiempo[5])+'.wav'

destino = 'K:/Proyecto T/Audios_server/'+ audio_name

with open(destino, 'wb') as f:
   ftp.retrbinary('RETR ' + file, f.write)

ftp.delete(file)

'250 DELE command successful'

In [8]:
destino1 = 'K:/Proyecto T/DATASETS/Screams/220659__marionagm90__scream.wav'
destino2 = 'K:/Proyecto T/DATASETS/vs_release_16k/audio_16k/f0004_0_throatclearing.wav'

In [9]:
prediction = model_Prediction(destino1)
ipd.Audio(destino1)

1/1 [==============================] - 0s 100ms/step
El modelo a predicho la clase como  -->  Grito


In [18]:
if prediction == 'Llorar' or 'Grito':
    mensaje = client.messages.create(to='whatsapp:+56990220976', from_='whatsapp:+14155238886', body='Alerta desde dispositivo Esp32')

In [4]:
mensaje = client.messages.create(to='whatsapp:+56990220976', from_='whatsapp:+14155238886', body='prueba Alerta desde dispositivo Esp32 prueba 2')

In [ ]:
while True:
   try:
      archivos = ftp.nlst()
      num = len(archivos)

      if num == 0:
         print("Directorio del servidor vacio, esperando archivos wav")
         tm.sleep(5)
         continue
      else:
         file = archivos[0]
         
         tiempo = tm.localtime()
         audio_name = 'mic_'+str(tiempo[0])+str("%02d" % tiempo[1])+str("%02d" % tiempo[2])+'_'+str("%02d" % tiempo[3])+'_'+str("%02d" % tiempo[4])+'_'+str("%02d" % tiempo[5])+'.wav'
         
         destino = 'K:/Proyecto T/Audios_server/'+ audio_name

         with open(destino, 'wb') as f:
            ftp.retrbinary('RETR ' + file, f.write)

         ftp.delete(file)

         prediction = model_Prediction(destino)
         # play the file
         # ipd.Audio(destino)

         if prediction == 'Llorar' or 'Grito':
            mensaje = client.messages.create(to='whatsapp:+56990220976', 
                                             from_='whatsapp:+14155238886', 
                                             body='Alerta desde dispositivo Esp32')
            print(prediction+" identificado, mensaje enviado")
         else:
            print(prediction+" identificado, sin acción nesesaria")
   except:
      print("Error encontrado en ciclo while")
      break

In [ ]:
SERVER = 'ftp-serveralways.alwaysdata.net'
USER = 'serveralways'
USERPASS = 'Always322981'

SID = 'ACdd02d37c618e196bf7d73790984b4797'
TOKEN = 'bfbc7af7aacbfd63da6470601b06d383'

WSP_TO = '+56990220976'
WSP_FROM = '+14155238886'

In [ ]:
from twilio.rest import Client
sid = SID
authToken = TOKEN
client = Client(sid, authToken)

model = load_model('Model.h5')

final = pd.read_pickle("extracted_df.pkl")
y = np.array(final["class"].tolist())
le = LabelEncoder()
to_categorical(le.fit_transform(y))

ftp = ftplib.FTP(SERVER, USER, USERPASS)
ftp.pwd()
ftp.cwd("audios")
ftp.retrlines('LIST')

while True:
   try:
      archivos = ftp.nlst()
      num = len(archivos)

      if num == 0:
         print("Directorio del servidor vacio, esperando archivos wav")
         tm.sleep(5)
         continue
      else:
         file = archivos[0]

         tiempo = tm.localtime()
         audio_name = 'mic_'+str(tiempo[0])+str("%02d" % tiempo[1])+str("%02d" % tiempo[2])+'_'+str("%02d" % tiempo[3])+'_'+str("%02d" % tiempo[4])+'_'+str("%02d" % tiempo[5])+'.wav'
         
         destino = 'K:/Proyecto T/Audios_server/'+ audio_name

         with open(destino, 'wb') as f:
            ftp.retrbinary('RETR ' + file, f.write)

         ftp.delete(file)

         prediction = model_Prediction(destino)
         # play the file
         # ipd.Audio(destino)

         if prediction == 'Llorar' or 'Grito':
            mensaje = client.messages.create(to='whatsapp:'+WSP_TO, 
                                             from_='whatsapp:'+WSP_FROM, 
                                             body='Alerta desde dispositivo Esp32')
            print(prediction+" identificado, mensaje enviado")
         else:
            print(prediction+" identificado, sin acción nesesaria")
   except:
      print("Error encontrado en ciclo while")
      break